## Imports

In [ ]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np
import os
import numpy as np
from sympy import Matrix
import pytest
from attr import define, field
from sympy import MutableDenseNDimArray as Arr


from library.python.fvm.solver_numpy import Settings
from library.model.models.shallow_water import ShallowWaterEquations
from library.model.basemodel import Model
import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
from library.python.misc.misc import Zstruct
import library.dg.firedrake_solver as dg


ModuleNotFoundError: No module named 'firedrake'

In [ ]:
@define(frozen=True, slots=True, kw_only=True)
class SWE(Model):
    dimension: int = 1
    variables: Zstruct = field(init=False, default=dimension + 1)
    aux_variables: Zstruct = field(default=1)
    _default_parameters: dict = field(
        init=False, factory=lambda: {"g": 9.81, "ex": 0.0, "ey": 0.0, "ez": 1.0}
    )

    def project_2d_to_3d(self):
        out = Arr.zeros(6)
        dim = self.dimension
        x = self.position[0]
        y = self.position[1]
        z = self.position[2]
        b = 0
        h = self.variables[0]
        U = [hu / h for hu in self.variables[1 : 1 + dim]]
        rho_w = 1000.0
        g = 9.81
        out[0] = b
        out[1] = h
        out[2] = U[0]
        out[3] = 0 if dim == 1 else U[1]
        out[4] = 0
        out[5] = rho_w * g * h * (1 - z)
        return out

    def flux(self):
        dim = self.dimension
        h = self.variables[0]
        U = Matrix([hu / h for hu in self.variables[1 : 1 + dim]])
        g = self.parameters.g
        I = Matrix.eye(dim)
        F = Matrix.zeros(self.variables.length(), dim)
        F[0, :] = (h * U).T
        F[1:, :] = h * U * U.T + g / 2 * h**2 * I
        return Arr([F[:, d] for d in range(dim)])


# Transformation to UFL Code (Medium)

### Map from Sympy to UFL

In [ ]:
bcs = BC.BoundaryConditions(
    [
        BC.Extrapolation(physical_tag="top"),
        BC.Extrapolation(physical_tag="bottom"),
        BC.Extrapolation(physical_tag="left"),
        BC.Extrapolation(physical_tag="right"),
        # BC.Wall(physical_tag="top"),
        # BC.Wall(physical_tag="bottom"),
        # BC.Wall(physical_tag="left"),
        # BC.Wall(physical_tag="right"),
    ]
)

 ### Initial condition
def ic_q(x):
    R = 0.15
    r = np.sqrt((x[0] - 0.7)**2 + (x[1] - 0.7)**2)
    b = 0.1*np.sqrt((x[0] - 3.)**2 + (x[1] - 3.)**2)
    return np.array([0.*x[0], np.where(r <= R, 1., 0.9), 0.*x[0], 0.*x[0]])

ic = IC.UserFunction(ic_q)

model = ShallowWaterEquations(
    dimension=2,
    boundary_conditions=bcs,
    initial_conditions=ic,
)

settings = Settings(name="Firedrake", output=Zstruct(directory="outputs/firedrake", filename='dg'))


2025-10-13 06:46:11.257 | WARNING  | library.python.misc.misc:__init__:146 - No 'clean_directory' attribute found in output Zstruct. Default: False


In [3]:
solver = dg.DolfinxHyperbolicSolver(settings=settings, time_end = 10.)

In [4]:
main_dir = os.getenv('ZOOMY_DIR')
path_to_mesh = os.path.join(main_dir, "meshes", "channel_quad_2d", "mesh_coarse.msh")
solver.solve(path_to_mesh, model)


Info    : Reading '/home/ingo/Git/Zoomy/meshes/channel_quad_2d/mesh_coarse.msh'...
Info    : 729 nodes
Info    : 816 elements
Info    : Done reading '/home/ingo/Git/Zoomy/meshes/channel_quad_2d/mesh_coarse.msh'


Setup  sim , solving PDE: 100%|██████████| 10.009173603152462/10.0 [00:15<00:00,  1.58s/it] 


Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, quadrilateral, 1, equispaced, unset, False, float64, []), (2,)), 0), blocked element (Basix element (P, quadrilateral, 0, gll_warped, unset, True, float64, []), (3,))), 1)